In [ ]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
pd.set_option('display.max_colwidth', None)

## Get the paths and build table names for iteration

In [4]:
data_path = '../src/data'
mapping_corpus_path = data_path + r'/product/lspc2020_to_tablecorpus'
mapping_corpus_path_2 = data_path + r'/product/lspc2020_to_tablecorpus/Cleaned'
table_corpus_path = data_path + r'/product/product_top100/cleaned'
table_corpus_path_with_id = data_path + r'/product/product_top100/cleaned/with_id'
table_corpus_path2 = data_path + r'/product/product_minimum3/cleaned/with_id'

In [5]:
zip_files_mapping = [file for file in os.listdir(mapping_corpus_path_2) if file.endswith('.json.gz')]
zip_files_tables = [file for file in os.listdir(table_corpus_path) if file.endswith('.json.gz')]

## Match the number dictionaries with the information about the brand

In [6]:
df_joined_electronics = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_electronics'), compression='gzip', orient='records', lines=True)

In [7]:
df_joined_electronics

,cluster_id,url,row_id,table_id,Valid,brand
0,59178314,https://www.cultureindoor.com/796-irrigation-a...,1561,Product_cultureindoor.com_September2020.json.gz,1,None
1,70692685,https://www.culturekings.com/products/new-era-...,3893,Product_culturekings.com_September2020.json.gz,1,None
2,395606,https://www.culturekings.com/products/new-era-...,3894,Product_culturekings.com_September2020.json.gz,1,None
3,3361797,https://www.customizedgirl.com/design/2605608/...,9318,Product_customizedgirl.com_September2020.json.gz,1,None
4,235755,https://www.customizedgirl.com/design/2613423/...,9319,Product_customizedgirl.com_September2020.json.gz,1,None
...,...,...,...,...,...,...
24687502,8653717,https://shop-list.com/women/utuwa-tatara/sg323...,1620,Product_shop-list.com_September2020.json.gz,1,None
24687503,22258637,https://shop-list.com/women/youmotto/famille-ltm,1621,Product_shop-list.com_September2020.json.gz,1,None
24687504,41434658,https://shop.agwaycapecod.com/products/0190147...,126,Product_agwaycapecod.com_September2020.json.gz,1,None
24687505,62702145,https://shop.agwaycapecod.com/products/0373214...,127,Product_agwaycapecod.com_September2020.json.gz,1,None


In [8]:
df_joined_clothes.to_json(mapping_corpus_path_2 + '/joined_clothes', compression='gzip', orient='records', lines=True)

NameError: name 'df_joined_clothes' is not defined

In [ ]:
df_joined_clothes = pd.read_json(os.path.join(mapping_corpus_path_2, 'joined_clothes'), compression='gzip', orient='records', lines=True)
df_joined_clothes

In [ ]:
# df_tables_count=df_join.groupby('cluster_id').count()
# df_tables_count[df_tables_count['brand']>2]

## Cluster statistics for product category electronics

In [9]:
df_grouped_electronics = df_joined_electronics.groupby('cluster_id').count()

In [10]:
# only look at clusters that have at least one brand associated
df_set_electronics = df_grouped_electronics[df_grouped_electronics['brand']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [11]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_electronics=df_set_electronics[df_set_electronics['Amount']>1]
df_set_electronics

,cluster_id,Amount
0,419,6
3,968,4
4,985,14
5,997,3
6,1040,21
...,...,...
104491,80393061,2
104517,80418008,3
104529,80434638,4
104532,80436986,2


In [17]:
df_15_electronics=df_set_electronics[df_set_electronics['Amount']>15]
df_15_electronics

,cluster_id,Amount
6,1040,21
25,6076,19
26,6443,25
28,6505,16
29,6690,20
...,...,...
100248,76869317,60
101968,78260404,80
103569,79512161,16
104156,80097153,60


In [23]:
#merge brand name to cluster amount
df_cluster_brand = df_15_electronics[df_15_electronics['Amount']<400].merge(df_joined_electronics.dropna()[['cluster_id','brand']].drop_duplicates('cluster_id', keep='last'), left_on=['cluster_id'], right_on = ['cluster_id'], how='left')
df_cluster_brand

,cluster_id,Amount,brand
0,1040,21,nikon
1,6076,19,canon
2,6443,25,canon
3,6505,16,apple
4,6690,20,cisco
...,...,...,...
328,76869317,60,samsung
329,78260404,80,samsung
330,79512161,16,xiaomi
331,80097153,60,samsung


In [24]:
#get the top clusters per brand
df_top_clusters = df_cluster_brand.sort_values(['Amount'], ascending=False).drop_duplicates(subset=["brand"], keep="first")
df_top_clusters

,cluster_id,Amount,brand
41,184777,314,sony
158,1566090,177,canon
219,4494482,83,samsung
189,2456244,55,nikon
80,491900,38,apple
10,28307,36,hp
110,835420,30,amd
43,197396,30,intel
200,2848082,29,asus
7,14418,25,panasonic


In [82]:
df_joined_electronics[(df_joined_electronics['cluster_id']==1566090)]

,cluster_id,url,row_id,table_id,Valid,brand,product_name
105918,1566090,https://www.scottsbikes.com/product/park-tool-chain-wear-indicator-222236-1.htm,222,Product_scottsbikes.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm
541872,1566090,https://www.bigshark.com/product/park-tool-chain-wear-indicator-222236-1.htm,2221,Product_bigshark.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm
968763,1566090,https://avcstore.com/products/canon-mt-26ex-rt-macro-twin-lite,303,Product_avcstore.com_September2020.json.gz,1,canon,canon-mt-26ex-rt-macro-twin-lite
1092099,1566090,https://www.trekstoreracine.com/product/park-tool-chain-wear-indicator-222236-1.htm,144,Product_trekstoreracine.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm
1631174,1566090,https://www.seabasscycles.co.uk/cc-32-chain-wear-indicator.html,75,Product_seabasscycles.co.uk_September2020.json.gz,1,None,cc-32-chain-wear-indicator.html
...,...,...,...,...,...,...,...
24134777,1566090,https://www.seriouscycles.com/product/park-tool-chain-wear-indicator-222236-1.htm,1717,Product_seriouscycles.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm
24382426,1566090,https://www.freewheelbike.com/product/park-tool-chain-wear-indicator-222236-1.htm,625,Product_freewheelbike.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm
24398829,1566090,https://www.actionsportsbicyclecenter.com/product/park-tool-chain-wear-indicator-222236-1.htm,462,Product_actionsportsbicyclecenter.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm
24427409,1566090,https://www.bikesportbikes.com/product/park-tool-chain-wear-indicator-222236-1.htm,1354,Product_bikesportbikes.com_September2020.json.gz,1,None,park-tool-chain-wear-indicator-222236-1.htm


In [27]:
#get a new column with the product names from the url
df_joined_electronics['product_name'] = df_joined_electronics['url'].apply(lambda row: row.split("/")[-1])

In [28]:
df_joined_electronics

,cluster_id,url,row_id,table_id,Valid,brand,product_name
0,59178314,https://www.cultureindoor.com/796-irrigation-arrosage-tuyau-pe-20mm-semi-rigide-rouleau-100m.html,1561,Product_cultureindoor.com_September2020.json.gz,1,None,796-irrigation-arrosage-tuyau-pe-20mm-semi-rigide-rouleau-100m.html
1,70692685,https://www.culturekings.com/products/new-era-new-era-new-orleans-pelicans-nba-940-a-frame-snapback-black,3893,Product_culturekings.com_September2020.json.gz,1,None,new-era-new-era-new-orleans-pelicans-nba-940-a-frame-snapback-black
2,395606,https://www.culturekings.com/products/new-era-x-nrl-west-tigers-aframe-charcoal-orange,3894,Product_culturekings.com_September2020.json.gz,1,None,new-era-x-nrl-west-tigers-aframe-charcoal-orange
3,3361797,https://www.customizedgirl.com/design/2605608/The+Real+Baseball+Moms,9318,Product_customizedgirl.com_September2020.json.gz,1,None,The+Real+Baseball+Moms
4,235755,https://www.customizedgirl.com/design/2613423/Mentally+Dating+Ronaldo,9319,Product_customizedgirl.com_September2020.json.gz,1,None,Mentally+Dating+Ronaldo
...,...,...,...,...,...,...,...
24687502,8653717,https://shop-list.com/women/utuwa-tatara/sg3232-03,1620,Product_shop-list.com_September2020.json.gz,1,None,sg3232-03
24687503,22258637,https://shop-list.com/women/youmotto/famille-ltm,1621,Product_shop-list.com_September2020.json.gz,1,None,famille-ltm
24687504,41434658,https://shop.agwaycapecod.com/products/019014710959/adult-small-breed-dog-food-5-lb,126,Product_agwaycapecod.com_September2020.json.gz,1,None,adult-small-breed-dog-food-5-lb
24687505,62702145,https://shop.agwaycapecod.com/products/037321455042/ant-killer-dust-3-lbs,127,Product_agwaycapecod.com_September2020.json.gz,1,None,ant-killer-dust-3-lbs


## Cluster statistics for product category clothes

In [29]:
df_grouped_clothes = df_joined_clothes.groupby('cluster_id').count()

In [30]:
# only look at clusters that have at least one brand associated
df_set_clothes = df_grouped_clothes[df_grouped_clothes['brand']>0].reset_index()[['cluster_id','table_id']].rename(columns={'table_id':'Amount'})

In [31]:
# We discard all clusters with less than 2 entries, cause we cannot match anything there, so 1,6 million clusters remain
df_set_clothes=df_set_clothes[df_set_clothes['Amount']>1]
df_set_clothes

,cluster_id,Amount
0,560,2
1,1507,3
2,2420,3
3,2523,2
4,2663,3
...,...,...
105590,80424345,2
105604,80438085,2
105623,80472255,4
105624,80472257,4


In [37]:
df_10_clothes=df_set_clothes[df_set_clothes['Amount']>10]
df_10_clothes

,cluster_id,Amount
11,5310,11
183,58043,12
246,79412,15
279,90549,12
319,102437,14
...,...,...
79674,58592784,19
88341,65475235,11
92112,68554513,13
103333,78110534,13


In [86]:
#merge brand name to cluster amount
df_cluster_brand_clothes = df_10_clothes[df_10_clothes['Amount']<400].merge(df_joined_clothes.dropna()[['cluster_id','brand']].drop_duplicates('cluster_id', keep='last'), left_on=['cluster_id'], right_on = ['cluster_id'], how='left')
df_cluster_brand_clothes

,cluster_id,Amount,brand
0,5310,11,armani
1,58043,12,burberry
2,79412,15,adidas
3,90549,12,louis vuitton
4,102437,14,chanel
...,...,...,...
100,58592784,19,nike
101,65475235,11,nike
102,68554513,13,nike
103,78110534,13,dolce & gabbana


In [87]:
#get the top clusters per brand
df_top_clusters_clothes = df_cluster_brand_clothes.sort_values(['Amount'], ascending=False).drop_duplicates(subset=["brand"], keep="first")
df_top_clusters_clothes

,cluster_id,Amount,brand
84,5672069,289,gucci
27,527917,110,chanel
62,1524820,95,prada
67,2132296,61,the north face
24,434734,53,adidas
71,2434654,46,louis vuitton
83,3324035,40,oakley
98,58083280,24,nike
78,2825229,24,calvin klein
47,944406,18,dolce & gabbana


In [96]:
df_joined_clothes[(df_joined_clothes['cluster_id']==22374915)]

,cluster_id,url,row_id,table_id,Valid,brand
1126480,22374915,https://06.aplikasionlineshop.com/index.php/product/unero-military-classical-backpack,14,Product_aplikasionlineshop.com_September2020.json.gz,1,None
2128782,22374915,https://pensalo.com/product/unero-military-classical-backpack,16,Product_pensalo.com_September2020.json.gz,1,None
5574153,22374915,https://nelau.com/?product=unero-military-classical-backpack,14,Product_nelau.com_September2020.json.gz,1,None
5848416,22374915,https://pcmundo.com.mx/producto/panasonic-invertr-900l-refrigerator,70,Product_pcmundo.com.mx_September2020.json.gz,1,None
6191685,22374915,https://ebonth.com/product/unero-military-classical-backpack,24,Product_ebonth.com_September2020.json.gz,1,None
6994438,22374915,https://winnershopbd.com/product/unero-military-classical-backpack,22,Product_winnershopbd.com_September2020.json.gz,1,None
8350514,22374915,https://ikotaonline.com/product/unero-military-classical-backpack,15,Product_ikotaonline.com_September2020.json.gz,1,None
8765168,22374915,https://grixbase.com/product/graco-slim-snacker-high-chair-whisk,26,Product_grixbase.com_September2020.json.gz,1,None
9447506,22374915,https://www.digitalshopup.com/product/unero-military-classical-backpack,6,Product_digitalshopup.com_September2020.json.gz,1,None
10271721,22374915,https://ktrworld.com/product/unero-military-classical-backpack,78,Product_ktrworld.com_September2020.json.gz,1,None


In [98]:
df_large[df_large['cluster_id']==22374915]

,cluster_id,url,row_id,table_id,Valid
1126480,22374915,https://06.aplikasionlineshop.com/index.php/product/unero-military-classical-backpack,14,Product_aplikasionlineshop.com_September2020.json.gz,1
2128782,22374915,https://pensalo.com/product/unero-military-classical-backpack,16,Product_pensalo.com_September2020.json.gz,1
5574153,22374915,https://nelau.com/?product=unero-military-classical-backpack,14,Product_nelau.com_September2020.json.gz,1
5848416,22374915,https://pcmundo.com.mx/producto/panasonic-invertr-900l-refrigerator,70,Product_pcmundo.com.mx_September2020.json.gz,1
6191685,22374915,https://ebonth.com/product/unero-military-classical-backpack,24,Product_ebonth.com_September2020.json.gz,1
6994438,22374915,https://winnershopbd.com/product/unero-military-classical-backpack,22,Product_winnershopbd.com_September2020.json.gz,1
8350514,22374915,https://ikotaonline.com/product/unero-military-classical-backpack,15,Product_ikotaonline.com_September2020.json.gz,1
8765168,22374915,https://grixbase.com/product/graco-slim-snacker-high-chair-whisk,26,Product_grixbase.com_September2020.json.gz,1
9447506,22374915,https://www.digitalshopup.com/product/unero-military-classical-backpack,6,Product_digitalshopup.com_September2020.json.gz,1
10271721,22374915,https://ktrworld.com/product/unero-military-classical-backpack,78,Product_ktrworld.com_September2020.json.gz,1
